# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_df = pd.read_csv('output_data/City.csv')
city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tasiilaq,65.6145,-37.6368,-0.53,53,0,4.21,GL,1647921312
1,1,Vila Velha,-20.3297,-40.2925,69.75,93,75,13.80,BR,1647921313
2,2,Te Anau,-45.4167,167.7167,72.41,49,52,2.95,NZ,1647921314
3,3,Mar del Plata,-38.0023,-57.5575,65.08,90,100,34.00,AR,1647921301
4,4,Le Port,-20.9373,55.2919,84.25,65,67,0.87,RE,1647921315


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"].astype(float)

fig = gmaps.figure(center = [0,0] ,zoom_level = 2)
 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
cities_weather_df = city_df.loc[(city_df['Max Temp'] >= 70) & (city_df['Max Temp'] <= 80) & 
                                  (city_df['Wind Speed'] <= 20) & (city_df['Humidity'] <= 30)]
cities_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
116,116,Ndélé,8.4109,20.6473,73.51,16,15,1.70,CF,1647921404
121,121,Biu,10.6129,12.1946,77.04,16,100,1.79,NG,1647921409
167,167,Cayenne,4.9333,-52.3333,78.10,1,0,5.75,GF,1647920798
366,366,Filingué,14.3521,3.3168,79.20,14,29,5.95,NE,1647921630
430,430,Biltine,14.5333,20.9167,75.25,10,54,7.23,TD,1647921696
505,505,Dali,25.7000,100.1833,72.09,27,19,7.11,CN,1647921756
529,529,Gao,16.6362,1.6370,75.58,9,10,10.83,ML,1647921732


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = cities_weather_df
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,116,Ndélé,8.4109,20.6473,73.51,16,15,1.70,CF,1647921404
1,121,Biu,10.6129,12.1946,77.04,16,100,1.79,NG,1647921409
2,167,Cayenne,4.9333,-52.3333,78.10,1,0,5.75,GF,1647920798
3,366,Filingué,14.3521,3.3168,79.20,14,29,5.95,NE,1647921630
4,430,Biltine,14.5333,20.9167,75.25,10,54,7.23,TD,1647921696
5,505,Dali,25.7000,100.1833,72.09,27,19,7.11,CN,1647921756
6,529,Gao,16.6362,1.6370,75.58,9,10,10.83,ML,1647921732


In [24]:
import json
import collections

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

for index, row in hotel_df.iterrows():

    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_name = requests.get(base_url, params=params)
    
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 10.5912904,
                    "lng": 12.2091283
                },
                "viewport": {
                    "northeast": {
                        "lat": 10.59254142989272,
                        "lng": 12.21032087989272
                    },
                    "southwest": {
                        "lat": 10.58984177010728,
                        "lng": 12.20762122010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
    

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 4.942839,
                    "lng": -52.299391
                },
                "viewport": {
                    "northeast": {
                        "lat": 4.944083129892721,
                        "lng": -52.29801597010727
                    },
                    "southwest": {
                        "lat": 4.941383470107278,
                        "lng": -52.30071562989271
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "icon_background_color": "#909CE1",
            "icon_mask_base_uri": "https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet",
            "name": "Grand Hotel Montabo",
            "opening_hours": {
                "open_now": true
            

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.
{
    "html_attributions": [],
    "next_page_token": "Aap_uEDQ_89D_ZfUG8y9J47Jyt0waUEUJIkQWgpD_CmRoUy1ye-XRjC45tVKxWTjJqSVBNzEN2poOk9PpCBsKXytVxqfU8vplgJfzB0TXTo7h9wDRjLgWKXlLlaSI5aCDCk4KfJTU2BtJbwPVg1gxa4qum-Uw5QLKblSv4hikPohdCuk8_E99CsPGicmaGqymlJfkBJN0UUvVDgfq3fQXupAWJyYfYffVWzB8SeyN-2W69yiknERwhhohQk7mGu0tFysUxY5ZV_DK8Zi_d809lrqw3qBUmnsPPaZad3osPc6IjUyZccJygpas_nx9YkrAR0SOcVQGVLNhKbGGFchawiz4unEMsuead_MQ0EC3vx_tfvQ1iTxCu8UK21Eaz2pvarkGVwxvzYC1O3FAj-Y3YI7odw11wvvYsY4Jd00e_CC2Mb0q7cw86w8zMes-cIX-4c1",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 25.687973,
                    "lng": 100.161
                },
                "viewport": {
         

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,116,Ndélé,8.4109,20.6473,73.51,16,15,1.70,CF,1647921404,NaN
1,121,Biu,10.6129,12.1946,77.04,16,100,1.79,NG,1647921409,Biu Luxury Motel and Suites
2,167,Cayenne,4.9333,-52.3333,78.10,1,0,5.75,GF,1647920798,Grand Hotel Montabo
3,366,Filingué,14.3521,3.3168,79.20,14,29,5.95,NE,1647921630,NaN
4,430,Biltine,14.5333,20.9167,75.25,10,54,7.23,TD,1647921696,NaN
5,505,Dali,25.7000,100.1833,72.09,27,19,7.11,CN,1647921756,The One Hotels & Resorts
6,529,Gao,16.6362,1.6370,75.58,9,10,10.83,ML,1647921732,NaN


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

fig